In [1]:
import matplotlib 

matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

In [2]:
%run ./homogeneous_medium_sensors.ipynb

Sensors parameters:


/tmp/ipykernel_228745/2456891094.py:6: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In this notebooks, we will work on simulated measurements. To make things (a tiny bit) more realistic, we'll add some coloured noise to each sensor trace

In [3]:
import numpy as np
from jax import grad, random

from jwave.signal_processing import smooth

# Add colored noise
noise = random.normal(random.PRNGKey(42), sensors_data.shape)
for i in range(noise.shape[1]):
    noise = noise.at[:, i].set(smooth(noise[:, i]))

noisy_data = sensors_data + 0.2 * noise

# Show noisy traces
_field = FourierSeries(noisy_data.T, domain)
show_field(_field, "Noisy acoustic signals")
plt.xlabel("Time step")
plt.ylabel("Sensor position")
plt.axis("on")
plt.show()

/tmp/ipykernel_228745/1280918191.py:19: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


## Automatic differentiation

In `jwave`, it is possible to take the gradient with respect to any scalar loss, as shown in the following example. The gradients will have the same datatypes as the inputs of to the function, so `Field` gradients will be mapped to `Field` objects.

Here, we write a simple time reversal algorithm using autodiff. Note that time-reversal is proportional to the derivative operator of the `MSE` loss with respect to the measurement data, applied with zero initial conditions.

In [4]:
# The following uses the reciprocity of the wave equation to generate
# a time reversal imaging algorithm


def solver(p0):
    return simulate_wave_propagation(medium, time_axis, p0=p0, sensors=sensors)


@jit  # Compile the whole algorithm
def lazy_time_reversal(measurements):

    # Mask out elements outside the sensors ring
    mask = _circ_mask(N, 39, (64, 64))
    mask = np.expand_dims(mask, -1)

    def mse_loss(p0, measurements):
        p0 = p0.replace_params(p0.params * mask)
        p_pred = solver(p0)[..., 0]
        return 0.5 * jnp.sum(jnp.abs(p_pred - measurements) ** 2)

    # Start from an empty field
    p0 = FourierSeries.empty(domain)

    # Take the gradient of the MSE loss w.r.t. the
    # measured data
    p_grad = grad(mse_loss)(p0, measurements)

    return -p_grad


# Reconstruct initial pressure distribution
recon_image = lazy_time_reversal(noisy_data)

# Show reconstructed field
show_field(recon_image, "Reconstructed initial pressure using autograd")

In [5]:
# Timings for the reconstruction algorithm, should be
# ~ 2x the forward function.
%timeit lazy_time_reversal(noisy_data).params.block_until_ready()

86.9 ms ± 2.17 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
# Save for paper
fig, ax = plt.subplots(1,3, figsize=(13,3), dpi=100)

im1 = ax[0].imshow(p0.on_grid, cmap="RdBu_r", vmin=-2, vmax=2)
cbar = fig.colorbar(im1, ax=ax[0])
ax[0].axis('off')
ax[0].set_title('Initial pressure')
ax[0].scatter(x, y, label="sensors")
ax[0].legend()

im2 = ax[1].imshow(noisy_data.T, cmap="RdBu_r", vmin=-2, vmax=2, aspect='auto')
cbar = fig.colorbar(im2, ax=ax[1])
ax[1].set_title('Noisy traces')
ax[1].set_ylabel("Sensor num.")
ax[1].set_xlabel("Time-step")

im1 = ax[2].imshow(recon_image.on_grid, cmap="RdBu_r", vmin=-0.6, vmax=0.6)
cbar = fig.colorbar(im1, ax=ax[2])
ax[2].axis('off')
ax[2].set_title('Recovered initial pressure')

plt.savefig("initial_pressure_recon.pgf", bbox_inches="tight")